<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Máquinas de Aprendizaje II-2018 </h1>

<H3 align='center'> Tarea 3: Pregunta 2 </H3>
<hr style="height:2px;border:none"/>

## Integrantes

* Andrés Huerta - 201473544-8 - andres.huerta.14@sansano.usm.cl
* Felipe Vega - 201473511-1 - felipe.vega.14@sansano.usm.cl



<hr style="height:2px;border:none"/>


In [1]:
#Imports Here :D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import *
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


<hr style="height:2px;border:none"/>
# Redes Convolucionales sobre imágenes
<hr style="height:2px;border:none"/>

> **a) Construya funciones para leer los datos y cargarlos al momento de entrenar (durante cada *epoch*), para ésto utilice *Image Data Generator* de keras.**

En caso de querer ejecutar el cuaderno, para acelerar la ejecución se guardaron los modelos ya entrenados en archivos aparte.

<font size=5> [Link a modelos ya entrenados](https://drive.google.com/file/d/10KWWkXUkebwefi8ZMssGPaK3uWIfIwQm/view?usp=sharing) </font>

Al descargar, por favor descomprimir en la misma carpeta que este cuaderno.



In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255) #no transformation

train_generator = train_datagen.flow_from_directory(
        'food_data/train',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32)

validation_generator = test_datagen.flow_from_directory(
        'food_data/val',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


Se generan los sets de entrenamiento y testing, para el set de entrenamiento se cuenta con un total de 2400 imagenes repartidas equitativamente entre hotdogs, hamburguesas y pizzas. El test de testing a su vez cuenta con 600 imagenes donde cada clase tiene un total de 200 imagenes.

Analizando algunas de las imagenes individualmente en la carpeta contenedora se nota que existen algunas imagenes que no muestran los elementos principales o en algunas ocasiones estas aparecen muy pequeñas, dichas imagenes podrían quizas ser ruido para el modelo a entrenar.

> **b) Utilice la red tradicional (*Feed Forward*) entregada en el código para ser entrenada sobre los datos vectorizados, esto es que cada imagen queda representada como un vector gigante, y las 3 clases a las que se enfrenta. Evalúe el modelo con la métrica *accuracy* sobre el conjunto de validación.**

In [3]:
def do_FFN():
    model = Sequential()
    model.add(Flatten(input_shape=train_generator.image_shape)) #full dense
    model.add(BatchNormalization()) #to normalize the input..
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128,activation='relu')) #128
    model.add(Dropout(0.25))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # Let's train the model using RMSprop
    model.summary()
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator.classes)//train_generator.batch_size, #samples//batch_size
        epochs=25,
        validation_data=validation_generator,
        validation_steps=len(validation_generator.classes)//validation_generator.batch_size)
    return model

Se utiliza una red neuronal FF para clasificación con un total de 3 capas, siendo solo 1 la capa oculta.

Debido a que se esta tratando con un problema de clasificación multiple exclusivo se utiliza como función de perdida catergorical_crossentropy.

ademas, para evitar el re-entrenamiento si se borra el modelo se guarda localmente.

In [4]:
ffn = load_model("fnn.h5")

En caso de querer entrenarlo nuevamente descomentar esta sección.

In [34]:
#ffn = do_FFN()
#ffn.save("fnn.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 67500)             0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 67500)             270000    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               17280256  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 387       
Total para

In [5]:
print("Accuracy validation: ",ffn.evaluate_generator(generator=validation_generator)[1])

Accuracy validation:  0.4849999996026357


Se obtiene un accuracy de 0.48 con el set de testing. Una manera de mejorar este score podría ser limpiar el dataset de entrenamiento, por ejemplo eliminando aquellas imagenes que no presentan la comida de manera reconocible. La ingeniería de atributos es la clave!

Otra forma de mejorar el desempeño, sería sintonizar los parametros por medio de una validación cruzada, se podría ademas intentar comprobar el desempeño con otro optimizador, pero debido al alto uso de GPU esta opción podría demorar mucho tiempo (el cual lamentablemente en este periodo no disponemos :c).

>** c) Utilice la red convolucional (*CNN*) entregada en el código para ser entrenada sobre los datos brutos, matrices RGB de píxeles, y las 3 clases a las que se enfrenta. Evalúe el modelo con la métrica *accuracy* sobre el conjunto de validación. Compare.**

In [6]:
def do_CNN(train_generator):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=train_generator.image_shape,activation='relu'))
    model.add(Conv2D(32, (3, 3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
    model.add(Conv2D(64, (3, 3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5)) 
    model.add(Dense(len(train_generator.class_indices),activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # Let's train the model using RMSprop
    model.summary()
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator.classes)//train_generator.batch_size, #samples//batch_size
        epochs=25,
        validation_data=validation_generator,
        validation_steps=len(validation_generator.classes)//validation_generator.batch_size)
    return model

En este caso se genera una red de convolución, la cual consiste en separar las matrices de las imagenes en matrices de menor tamaño (dividir y conquistar) el trade-off que se obtiene de esta aplicación es que, en general, el desempeño del modelo debería aumentar, pero a cambio de un mayor costo de ejecución (cada epoch de esta parte demora un total de 5 minutos o más en completarse).

El optimizador utilizado nuevamente corresponde a rmsprop, el cual de acuerdo a la documentación es efectivo cuando se esta trabajando con modelos recurrentes.

La red generada posee 4 capas convolucionales las cuales subdividen el problema en zancadas de 3x3, al final de la red se agrupan todas las redes formadas y se trabaja como una red ff.

Se entrena la red y nuevamente se guarda la red localmente.

In [7]:
cnn = load_model("red_convolucion.h5")

In [11]:
#cnn = do_CNN(train_generator)
#cnn.save("red_convolucion.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
__________

Se utiliza la metrica accuracy_score

In [8]:
print("CNN:")
print("Accuracy validation: ",cnn.evaluate_generator(generator=validation_generator)[1])

print("FFN:")
print("Accuracy validation: ",ffn.evaluate_generator(generator=validation_generator)[1])

CNN:
Accuracy validation:  0.696666665871938
FFN:
Accuracy validation:  0.4849999992052714


Como se comento anteriormente una red de convolución tiende a mejorar el desempeño para modelos que utilicen matrices de gran tamaño, en este caso se ve una clara mejora de mas de 0.2 puntos sobre el set de testing!

> **d) Genere un conjunto datos con incorrecta etiquetación de manera manual y vea si el modelo convolucional se sigue comportando de la misma manera. Para esto tome 100 imágenes aleatorias de entrenamiento de la carpeta *hot dog* y 100 imágenes aleatorias de entrenamiento de la carpeta *hamburger* e intercambielas, sin manipular las imágenes de la carpeta *pizza* y con el conjunto de validación intacto. Genere las matrices de confusión en el conjunto de validación para visualizar cómo afectó al modelo la corrupción realizada a los datos.**

Se creó una nueva carpeta con 100 de las imagenes de hot dogs con 100 imagenes de hamburguesas y se genera el nuevo conjunto de entrenamiento.

In [9]:
train_generator2 = train_datagen.flow_from_directory(
        'food_data/train2',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32)

Found 2400 images belonging to 3 classes.


Se reentrena la red, pero con los datos sucios.

In [10]:
cnn_noise = load_model("convolucion_noise.h5")

In [10]:
#cnn_noise = do_CNN(train_generator2)
#cnn_noise.save("convolucion_noise.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
__________

Se realiza una comparación simple del desempeño de cada uno de los modelos.

In [11]:
print("CNN limpia:")
print("Accuracy validation: ",cnn.evaluate_generator(generator=validation_generator)[1])

print("CNN sucia:")
print("Accuracy validation: ",cnn_noise.evaluate_generator(generator=validation_generator)[1])

CNN limpia:
Accuracy validation:  0.6966666666666667
CNN sucia:
Accuracy validation:  0.6716666666666666


Realizando una metrica de desempeño simple se puede ver que la calidad del modelo de la red de convolución sucia, disminuye su magnitud, aun asi, la diferencia es de un poco mas de 0.02 puntos lo cual es sorprendente considerando que se esta entrenando erroneamente el 12,5% de los datos de hotdogs y hamburguesas.

Se utiliza una matriz de confusión para ver más a fondo el comportamiento de las redes convolucionales entrenadas.

In [ ]:
Y_pred_noise = cnn_noise.predict_generator(validation_generator,len(validation_generator.classes)//validation_generator.batch_size+1)
y_pred_noise = np.argmax(Y_pred_noise, axis=1)

Y_pred = cnn.predict_generator(validation_generator,len(validation_generator.classes)//validation_generator.batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

Y_pred_ffn = ffn.predict_generator(validation_generator,len(validation_generator.classes)//validation_generator.batch_size+1)
y_pred_ffn = np.argmax(Y_pred_ffn, axis=1)

Se crea ademas una matriz de confusión para la red ff, solo para comparar el comportamiento.

In [ ]:
confusion_m = confusion_matrix(validation_generator.classes, y_pred)
confusion_mn = confusion_matrix(validation_generator.classes, y_pred_noise)
confusion_mf = confusion_matrix(validation_generator.classes, y_pred_ffn)

In [ ]:
print("FF:")
confusion_mf

Se puede ver que la red feed-forward presenta problemas para clasificar la primera clase, clasificando correctamente solo el 8% de los datos de testing de dicha clase, la mejor clasificación que realiza corresponde a la tercera clase, alcanzando un poco mas del 50% de precisión.

In [ ]:
print("CNN limpio: ")
confusion_m

Para la red de convolución con los datos limpios se puede ver una mejora considerable para la primera clase, subiendo de un 8% de prcisión a un 35%, la precisión de la segunda clase también aumento, pero la tercera clase presenta una perdida de calidad.

In [ ]:
print("CNN sucio")
confusion_mn

Finalmente, se visualiza la matriz de confusión de la red de convolución con datos sucios, y se puede notar que la gran diferencia entre los resultados esta presente en la distribución de las imagenes clasificadas incorrectamente, a pesar de que el desempeño para las correctas clasificaciones disminuye se puede notar que no es una diferencia muy grande, lo cual resulta bastante interesante.

Analizando el desempeño de la tercera posibilidad de clasificación se puede asumir que dicha clase corresponde a las pizzas, puesto que su desempeño se vio afectado en menor medida, tan solo 1 dato de diferencia.

De entre todos los modelos entrenados, el que presento mejor desempeño corresponde a la red de convolución no corrupta, aun así la diferencia entre el modelo entrenado con datos correctos vs datos corruptos es de 0.02 puntos de presición lo cual resulta sorprendente tomando en cuenta que 12,5% de los datos de entrenamiento son erroneos.